In [1]:
import numpy as np

In [68]:
# filename = "AoC day 8 example data.txt"
filename = "AoC day 8 data.txt"
with open(filename, 'r') as f:
    data = [list(line.strip()) for line in f.readlines()]
data = np.array(data).astype(int)
data

array([[1, 2, 2, ..., 2, 0, 1],
       [1, 2, 2, ..., 2, 1, 1],
       [0, 0, 2, ..., 0, 2, 0],
       ...,
       [1, 2, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 1, 0],
       [1, 2, 1, ..., 1, 1, 0]])

In [69]:
def visible_from_left(heights: np.ndarray) -> np.ndarray:
    """Returns a mask for trees visible from the left."""
    level = -1
    visible = np.zeros_like(heights)
    for i in range(len(heights)):
        if heights[i] > level:
            visible[i] = 1
            level = heights[i]
    return visible

def visible_as_rows(heights: np.ndarray) -> np.ndarray:
    """Returns a mask for trees visible as rows."""
    visible = []
    for row in heights:
        visible_l_to_r = visible_from_left(row)
        visible_r_to_l = visible_from_left(row[::-1])[::-1]
        visible.append(np.bitwise_or(visible_l_to_r, visible_r_to_l))
    return np.vstack(visible)

In [70]:
# calculate the number of "visible" trees
# a tree is considered visible if it is:
# (a) at an edge or corner.
# (b) taller than all other trees before it in a column or row.
# plan: do this as a mask, to avoid double counting?
# now check each row and column (from both sides) to find visible trees
# use row-wise check twice, but transpose the data for column-wise checking
visible = np.bitwise_or(visible_as_rows(data), visible_as_rows(data.T).T)
print(visible)
print(visible.sum())   

[[1 1 1 ... 1 1 1]
 [1 1 0 ... 1 1 1]
 [1 0 1 ... 0 1 1]
 ...
 [1 1 0 ... 1 0 1]
 [1 0 0 ... 0 1 1]
 [1 1 1 ... 1 1 1]]
1812


In [72]:
# part 2
# calculate the scenic score for each tree
# for each tree how many other trees are visible along 
# each of the four directions?

def scenic_score(data):
    scores = np.empty_like(data)
    for i in range(data.shape[0]): # loop over rows
        for j in range(data.shape[1]): # loop over columns
            height = data[i, j]
            up = data[:i, j][::-1]
            down = data[i+1:, j]
            left = data[i, :j][::-1]
            right = data[i, j+1:]
            score = []
            for view in (up, down, left, right):
                blocked = np.argwhere(view >= height)
                if blocked.size > 0:
                    score.append(blocked[0][0] + 1)
                else:
                    score.append(len(view))
            scores[i, j] = np.product(score)
    return scores

In [73]:
scenic_score(data)

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  2,  5, ..., 32,  1,  0],
       [ 0,  1, 12, ...,  1, 12,  0],
       ...,
       [ 0, 12,  1, ...,  2,  1,  0],
       [ 0,  1,  1, ...,  1,  2,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [74]:
print(np.max(scenic_score(data)))

315495
